In [1]:
'''
2023/9/29 記載
・最終モデル
・次回の課題
→特徴量を増やす
→データ傾向の則ったnull埋めを行う ※今回は代表値を埋めただけ
→ベースモデルを作成することを意識する
→パラメーターチューニングをグリッドサーチで行う　※今回はoputunaを利用
→可能なら回帰でモデルを作成する

'''

'\n2023/9/29 記載\n・最終モデル\n・次回の課題\n→特徴量を増やす\n→データ傾向の則ったnull埋めを行う ※今回は代表値を埋めただけ\n→ベースモデルを作成することを意識する\n→パラメーターチューニングをグリッドサーチで行う\u3000※今回はoputunaを利用\n→可能なら回帰でモデルを作成する\n\n'

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
import sys
import math
import statistics
import japanize_matplotlib
import seaborn as sns

import os

from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

from graphviz import Digraph

graph_body = Digraph(format="png")
graph_body.attr('node', shape='box', fontname='MS Gothic')

# trainとtestの読み込み

In [3]:
train = pd.read_csv('../train.csv')
test = pd.read_csv('../test.csv')
alldata = pd.concat([train.copy(), test.copy()], axis = 0)

C:\Users\s-heya\AppData\Local\Temp\ipykernel_2132\1785753801.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('../train.csv')


In [4]:
print(len(train),len(test),len(alldata))

779404 18394 797798


# データ確認

# データ加工

In [5]:
# trainDev=train[train['取引価格（総額）_log'] >= 6].copy()
trainDev=train.copy()
testDev=test.copy()

In [6]:
# 市区町村コード

In [7]:
# 都道府県名
class_le = LabelEncoder()
alldata['class_都道府県名'] = class_le.fit_transform(alldata['都道府県名'])

def conv_isTOKYO(row):
    isTokyo = 0
    isKanagawa = 0
    if row['都道府県名'] == '東京都':
        isTokyo = 1
    if row['都道府県名'] == '神奈川県':
        isKanagawa = 1
    return(isTokyo, isKanagawa)
    
alldata[['is_東京都', 'is_神奈川県']] = alldata.apply(conv_isTOKYO, axis=1, result_type="expand")
alldata['class_都道府県名'] = class_le.fit_transform(alldata['都道府県名'])

In [8]:
# 市区町村名
# class_le = LabelEncoder()
# trainDev['class_市区町村名'] = class_le.fit_transform(trainDev['市区町村名'])

chikuCount = train['地区名'].value_counts()
shikuchosonCount = train['市区町村名'].value_counts()

def func(row):
    if row['市区町村名'] == row['市区町村名']:
        if shikuchosonCount[row['市区町村名']] >= 100:
            return row['市区町村名']
        else:
            return row['都道府県名']
    else:
        return row['都道府県名']

from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
alldata['conv_市区町村名'] = alldata.apply(func, axis=1)
alldata['class_市区町村名'] = class_le.fit_transform(alldata['conv_市区町村名'])

In [9]:
# 地区名
alldata['地区名'] = alldata['地区名'].fillna('不明')

class_le = LabelEncoder()
alldata['class_地区名'] = class_le.fit_transform(alldata['地区名'])

In [10]:
# 最寄駅：名称
alldata['最寄駅：名称'] = alldata['最寄駅：名称'].fillna('不明')

class_le = LabelEncoder()
alldata['class_最寄駅：名称'] = class_le.fit_transform(alldata['最寄駅：名称'])

In [11]:
#最寄駅：距離（分）
def normalize_moyori(moyori):
    if moyori == moyori:
        if moyori == '30分?60分':
            moyori = 45
        elif moyori == '1H?1H30':
            moyori = 75
        elif moyori == '1H30?2H':
            moyori = 105
        elif moyori == '2H?':
            moyori = 120
        moyori = int(float(moyori))
    return moyori
alldata['最寄駅：距離（分）'] = alldata['最寄駅：距離（分）'].apply(lambda x: normalize_moyori(x))

alldata['最寄駅：距離（分）'] = alldata['最寄駅：距離（分）'].fillna(alldata[:len(train)]['最寄駅：距離（分）'].mean())

In [12]:
alldata['間取り'] = alldata['間取り'].fillna('３ＬＤＫ')

#「間取り」派生
def madori_convert(row):
    is_plusS = 0
    is_plusK = 0
    madori = row['間取り']
    if madori == madori:
        #+S
        if madori.find('＋Ｓ') != -1:
            is_plusS = 1
            madori = madori.replace('＋Ｓ','')
        #+K
        if madori.find('＋Ｋ') != -1:
            is_plusK = 1
            madori = madori.replace('＋Ｋ','')
        
    return(int(is_plusS),int(is_plusK),madori)

alldata[['is_PlusS', 'is_PlusK', '間取り']] = alldata.apply(madori_convert, axis=1, result_type="expand")

class_le = LabelEncoder()
alldata['class_間取り'] = class_le.fit_transform(alldata['間取り'])

In [13]:
# 面積
import re
def normalize_menseki(area):
    if area == area:
        area = int(re.sub('m\^2未満|㎡以上', '', str(area)))
    
    if area > 180:
        area = 185
    
    return area

alldata['面積（㎡）'] = alldata['面積（㎡）'].apply(lambda x: normalize_menseki(x))

alldata['面積（㎡）'] = alldata['面積（㎡）'].fillna(alldata[:len(train)]['面積（㎡）'].mean())

In [14]:
#is_15m2
def conv_is_15m2(area):
    if area <= 15:
        return 1
    else:
        return 0

alldata['is_15m2'] = alldata['面積（㎡）'].apply(lambda x: conv_is_15m2(x))

In [15]:
#建築年
def convert_wareki_to_seireki(wareki):
    if wareki == wareki:
        if wareki == '戦前':
            wareki = '昭和20年'
        value = wareki[2:-1]
        if value == '元':
            value = 1
        else:
            value = int(value)
        if '昭和' in wareki:
            seireki = 1925+value
        elif '平成' in wareki:
            seireki = 1988+value
        elif '令和' in wareki:
            seireki = 2018+value
    else:
        seireki = wareki
    return seireki

alldata['建築年'] = alldata['建築年'].apply(lambda x: convert_wareki_to_seireki(x))

alldata['建築年'] = alldata['建築年'].fillna(statistics.mode(alldata[:len(train)]['建築年']))

In [16]:
#建物の構造(カテゴリ変数化する)
alldata['建物の構造'] = alldata['建物の構造'].fillna(statistics.mode(alldata[:len(train)]['建物の構造']))

class_le = LabelEncoder()
alldata['class_建物の構造'] = class_le.fit_transform(alldata['建物の構造'])

In [17]:
#用途(カテゴリ変数化)
alldata['用途'] = alldata['用途'].fillna(statistics.mode(alldata[:len(train)]['用途']))

class_le = LabelEncoder()
alldata['class_用途'] = class_le.fit_transform(alldata['用途'])

In [18]:
# 今後の利用目的(カテゴリ変数化)
alldata['今後の利用目的'] = alldata['今後の利用目的'].fillna('その他')

class_le = LabelEncoder()
alldata['class_今後の利用目的'] = class_le.fit_transform(alldata['今後の利用目的'])

In [19]:
# 都市計画(カテゴリ変数化)
alldata['都市計画'] = alldata['都市計画'].fillna('第１種中高層住居専用地域')

class_le = LabelEncoder()
alldata['class_都市計画'] = class_le.fit_transform(alldata['都市計画'])

In [20]:
# 建ぺい率（％）  
alldata['建ぺい率（％）'] = alldata['建ぺい率（％）'].fillna(alldata[:len(train)]['建ぺい率（％）'].median())

In [21]:
# 容積率（％）
alldata['容積率（％）'] = alldata['容積率（％）'].fillna(alldata[:len(train)]['容積率（％）'].median())

In [22]:
# 取引時点（取引年と取引月の2変数化）　※1月はじまりのため4月はじまりで直すべきかも
# 取引年
def convert_torihikiNen(torihikijiten):
    torihikiYear = torihikijiten[0:4]
    torihikiYear = int(float(torihikiYear))
    return torihikiYear
alldata['取引年'] = alldata['取引時点'].apply(lambda x: convert_torihikiNen(x))

# 取引月
def convert_torihikiTsuki(torihikijiten):
    torihikiTsuki = 1
    if torihikijiten[5:] == '第1四半期':
        torihikiTsuki = 1
    elif torihikijiten[5:] == '第2四半期':
        torihikiTsuki = 4
    elif torihikijiten[5:] == '第3四半期':
        torihikiTsuki = 7
    elif torihikijiten[5:] == '第4四半期':
        torihikiTsuki = 10
    return torihikiTsuki
alldata['取引月'] = alldata['取引時点'].apply(lambda x: convert_torihikiTsuki(x))

In [23]:
#改装(カテゴリ変数化)
alldata['都市計画'] = alldata['都市計画'].fillna('未改装')

class_le = LabelEncoder()
alldata['class_改装'] = class_le.fit_transform(alldata['改装'])

In [24]:
# 取引の事情等
alldata['取引の事情等'] = alldata['取引の事情等'].fillna('なし')

class_le = LabelEncoder()
alldata['class_取引の事情等'] = class_le.fit_transform(alldata['取引の事情等'])

In [25]:
# 建築経過年数
def convert_kenchikuYear(kenchikuYear):
    if kenchikuYear < 0:
        return 0
    elif kenchikuYear > 50:
        return 51
    else:
        return kenchikuYear
alldata['建築経過年数'] = alldata['取引年']-alldata['建築年'] 
alldata['建築経過年数'] = alldata['建築経過年数'].apply(lambda x: convert_kenchikuYear(x))

In [26]:
# 建築年×建築経過年
alldata['建築年_建築経過年'] = alldata['建築年'].astype(str) + '_' + alldata['建築経過年数'].astype(str)
class_le = LabelEncoder()
alldata['建築年_建築経過年'] = class_le.fit_transform(alldata['建築年_建築経過年'])

In [27]:
# 取引年代
alldata['取引年代'] = (alldata['取引年'] / 10).astype(int) * 10

In [28]:
# # 組み合わせ
# #https://assam-blog.com/python-basis-combination/

# from itertools import combinations as comb

# categoryClm = ['間取り', '建物の構造', '用途', '今後の利用目的', '都市計画', '取引の事情等']
# C = comb(categoryClm,2)

# L = list(C)

# len(L)

# for tp in L:
#     clmlist = list(tp)
#     print('{}_{}'.format(clmlist[0], clmlist[1]))
#     clmName = 'class_{}_{}'.format(clmlist[0], clmlist[1])
#     alldata[clmName] = alldata[clmlist[0]].astype(str) + '_' + alldata[clmlist[1]].astype(str)

In [29]:
alldata.columns

Index(['Unnamed: 0', 'ID', '種類', '地域', '市区町村コード', '都道府県名', '市区町村名', '地区名',
       '最寄駅：名称', '最寄駅：距離（分）', '間取り', '面積（㎡）', '土地の形状', '間口', '延床面積（㎡）', '建築年',
       '建物の構造', '用途', '今後の利用目的', '前面道路：方位', '前面道路：種類', '前面道路：幅員（ｍ）', '都市計画',
       '建ぺい率（％）', '容積率（％）', '取引時点', '改装', '取引の事情等', '取引価格（総額）_log',
       'class_都道府県名', 'is_東京都', 'is_神奈川県', 'conv_市区町村名', 'class_市区町村名',
       'class_地区名', 'class_最寄駅：名称', 'is_PlusS', 'is_PlusK', 'class_間取り',
       'is_15m2', 'class_建物の構造', 'class_用途', 'class_今後の利用目的', 'class_都市計画',
       '取引年', '取引月', 'class_改装', 'class_取引の事情等', '建築経過年数', '建築年_建築経過年',
       '取引年代'],
      dtype='object')

# モデル構築

In [30]:
# exp_val_columns = ['市区町村コード','最寄駅：距離（分）','面積（㎡）','建築年','建ぺい率（％）','容積率（％）',
#                    'class_都道府県名','class_地区名','class_最寄駅：名称','class_間取り','class_建物の構造',
#                    'class_用途','class_都市計画','取引年','取引月','class_改装','class_取引の事情等','class_今後の利用目的']
exp_val_columns = ['最寄駅：距離（分）','面積（㎡）','建ぺい率（％）','容積率（％）','建築年','取引年代',
                   'class_間取り','class_建物の構造','建築経過年数','is_PlusS', 'is_PlusK','is_東京都','is_神奈川県',
                   'class_用途','class_都市計画','取引年','取引月','class_改装','class_取引の事情等','class_今後の利用目的',
                   'is_15m2','市区町村コード']
obj_val_columns = ['取引価格（総額）_log']

train_for_model = alldata[:len(trainDev)].copy()
test_for_model = alldata[len(trainDev):]

train_for_model.head().to_csv('train_加工後.csv', index=False,encoding='cp932')

In [31]:
print(len(train_for_model),len(test_for_model))
print(test_for_model.columns)

779404 18394
Index(['Unnamed: 0', 'ID', '種類', '地域', '市区町村コード', '都道府県名', '市区町村名', '地区名',
       '最寄駅：名称', '最寄駅：距離（分）', '間取り', '面積（㎡）', '土地の形状', '間口', '延床面積（㎡）', '建築年',
       '建物の構造', '用途', '今後の利用目的', '前面道路：方位', '前面道路：種類', '前面道路：幅員（ｍ）', '都市計画',
       '建ぺい率（％）', '容積率（％）', '取引時点', '改装', '取引の事情等', '取引価格（総額）_log',
       'class_都道府県名', 'is_東京都', 'is_神奈川県', 'conv_市区町村名', 'class_市区町村名',
       'class_地区名', 'class_最寄駅：名称', 'is_PlusS', 'is_PlusK', 'class_間取り',
       'is_15m2', 'class_建物の構造', 'class_用途', 'class_今後の利用目的', 'class_都市計画',
       '取引年', '取引月', 'class_改装', 'class_取引の事情等', '建築経過年数', '建築年_建築経過年',
       '取引年代'],
      dtype='object')


In [32]:
from sklearn.model_selection import train_test_split

y_train =  train_for_model.copy()['取引価格（総額）_log']
X_train_base =  train_for_model.copy().drop('取引価格（総額）_log', axis=1)
X_test = test_for_model[exp_val_columns].copy()

# stratifyパラメータについて
# https://kakakakakku.hatenablog.com/entry/2021/11/30/160351
# random_stateを指定することで同じ分割を行う
X_train_base, X_valid_base, y_train, y_valid = \
    train_test_split(X_train_base, y_train, test_size=0.3,
                                 random_state=0)

X_train = X_train_base.copy()[exp_val_columns]
X_valid = X_valid_base.copy()[exp_val_columns]

In [33]:
X_train.columns

Index(['最寄駅：距離（分）', '面積（㎡）', '建ぺい率（％）', '容積率（％）', '建築年', '取引年代', 'class_間取り',
       'class_建物の構造', '建築経過年数', 'is_PlusS', 'is_PlusK', 'is_東京都', 'is_神奈川県',
       'class_用途', 'class_都市計画', '取引年', '取引月', 'class_改装', 'class_取引の事情等',
       'class_今後の利用目的', 'is_15m2', '市区町村コード'],
      dtype='object')

In [34]:
import time
import lightgbm as lgb

time_sta = time.perf_counter()
# params = {
#     'objective': 'regression_l1',
#     'metric': 'mae',
#     'random_state': 0,
#     'boosting_type': 'gbdt',
#     'verbose': -1,
#     'n_jobs': 8,
#     'force_col_wise': 'true'
# }

params = {'objective': 'regression_l1',
 'metric': 'l1',
 'random_state': 0,
 'boosting_type': 'gbdt',
 'verbose': -1,
 'n_jobs': 16,
 'force_col_wise': 'true',
 'max_depth': 7,
 'feature_pre_filter': False,
 'lambda_l1': 0.8934032275177914,
 'lambda_l2': 6.123046555632642e-05,
 'num_leaves': 51,
 'feature_fraction': 0.8999999999999999,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 20,
 'num_iterations': 10000,
 'early_stopping_round': 10
}

# categorical_features=['市区町村コード','class_都道府県名','class_地区名','class_最寄駅：名称','class_間取り','class_建物の構造',
#                    'class_用途','class_都市計画','取引年','取引月','class_改装','class_取引の事情等','class_今後の利用目的']
categorical_features=['class_間取り','class_建物の構造', 'class_間取り','is_PlusS', 'is_PlusK','is_東京都','is_神奈川県','取引年代',
                      'class_用途','class_都市計画','取引年','取引月','class_改装','class_取引の事情等','class_今後の利用目的',
                      'is_15m2','市区町村コード']

lgb_train = lgb.Dataset(X_train,y_train,categorical_feature=categorical_features)
lgb_eval  = lgb.Dataset(X_valid,y_valid,reference=lgb_train,categorical_feature=categorical_features)

# モデルの学習
model = lgb.train(
    params, 
    lgb_train, 
    valid_sets=[lgb_train,lgb_eval],
    num_boost_round=10000,
    callbacks=[lgb.early_stopping(stopping_rounds=10, 
                            verbose=True), # early_stopping用コールバック関数
                        lgb.log_evaluation(100)]
)

time_end = time.perf_counter()

print(time_end- time_sta)

C:\Users\s-heya\Anaconda3\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 10 rounds
[100]	training's l1: 0.0864285	valid_1's l1: 0.0883684
[200]	training's l1: 0.0823732	valid_1's l1: 0.0851853
[300]	training's l1: 0.0807069	valid_1's l1: 0.0840458
[400]	training's l1: 0.0797826	valid_1's l1: 0.0835065
[500]	training's l1: 0.0790856	valid_1's l1: 0.0831432
[600]	training's l1: 0.0785171	valid_1's l1: 0.0828965
[700]	training's l1: 0.0780167	valid_1's l1: 0.0826935
[800]	training's l1: 0.0775554	valid_1's l1: 0.082502
[900]	training's l1: 0.0771312	valid_1's l1: 0.0823426
[1000]	training's l1: 0.0767886	valid_1's l1: 0.0822226
[1100]	training's l1: 0.0764555	valid_1's l1: 0.0821039
[1200]	training's l1: 0.0761233	valid_1's l1: 0.0819795
[1300]	training's l1: 0.0758282	valid_1's l1: 0.0818834
[1400]	training's l1: 0.0755354	valid_1's l1: 0.0817707
[1500]	training's l1: 0.0752817	valid_1's l1: 0.0816851
[1600]	training's l1: 0.0750501	valid_1's l1: 0.0816137
[1700]	training's l1: 0.0748071	valid_1's l1: 0.08153

# モデル適用

In [35]:
y_pred = model.predict(X_test)
len(y_pred)

18394

In [36]:
sub = pd.read_csv('../sample_submission.csv')
sub['取引価格（総額）_log'] = y_pred
sub.to_csv('submission.csv', index=False)

# 評価

In [37]:
from sklearn.model_selection import train_test_split

y_train =  train_for_model.copy()['取引価格（総額）_log']
X_train_base =  train_for_model.copy().drop('取引価格（総額）_log', axis=1)
X_test = test_for_model[exp_val_columns].copy()

# stratifyパラメータについて
# https://kakakakakku.hatenablog.com/entry/2021/11/30/160351
# random_stateを指定することで同じ分割を行う
X_train_base, X_valid_base, y_train, y_valid = \
    train_test_split(X_train_base, y_train, test_size=0.3,
                                 random_state=42)

X_train = X_train_base.copy()[exp_val_columns]
X_valid = X_valid_base.copy()[exp_val_columns]

len(X_valid)
X_valid_base['取引価格（総額）_log'] = y_valid
X_valid_base['pred_取引価格（総額）_log'] = model.predict(X_valid)

from sklearn.metrics import mean_absolute_error
df = X_valid_base.copy()
print(mean_absolute_error(df['取引価格（総額）_log'], df['pred_取引価格（総額）_log']))

0.07336110568337847


In [38]:
importance = pd.DataFrame(
	model.feature_importance(importance_type='gain'),
	index=X_train.columns,
	columns=['importance']).sort_values('importance', ascending=False)

importance.astype(int)

,importance
市区町村コード,1987949
建築年,1703656
面積（㎡）,939762
取引年,455661
最寄駅：距離（分）,359047
建築経過年数,247894
class_改装,194159
class_都市計画,170110
class_間取り,153610
is_東京都,109032


In [39]:
importance = pd.DataFrame(
	model.feature_importance(importance_type='split'),
	index=X_train.columns,
	columns=['importance']).sort_values('importance', ascending=False)

importance

,importance
市区町村コード,49340
建築年,37776
取引年,37069
最寄駅：距離（分）,34274
建築経過年数,29875
面積（㎡）,28461
class_都市計画,21274
class_間取り,16257
class_改装,11674
容積率（％）,8510


In [40]:
# sys.exit()

In [41]:
# from sklearn.metrics import mean_absolute_error
# from sklearn.model_selection import KFold


# val_y_train = train_for_model['取引価格（総額）_log']
# val_X_train = train_for_model.drop('取引価格（総額）_log', axis=1)

# scores = []

# kf = KFold(n_splits = 4, shuffle=True, random_state=71)

# for tr_idx, va_idx in kf.split(val_X_train):
#     tr_x, va_x = val_X_train.iloc[tr_idx], val_X_train.iloc[va_idx]
#     tr_y, va_y = val_y_train.iloc[tr_idx], val_y_train.iloc[va_idx]
    
#     lgb_train = lgb.Dataset(tr_x,tr_y,categorical_feature=categorical_features)
#     lgb_eval  = lgb.Dataset(va_x,va_y,reference=lgb_train,categorical_feature=categorical_features)
    
#     # モデルの学習
#     model = lgb.train(
#         params, 
#         lgb_train, 
#         valid_sets=[lgb_train,lgb_eval],
#         num_boost_round=10000,
#         callbacks=[lgb.early_stopping(stopping_rounds=10, 
#                                 verbose=True), # early_stopping用コールバック関数
#                             lgb.log_evaluation(100)]
#     )
    
#     ev_y_pred = model.predict(va_x)
    
#     score = mean_absolute_error(va_y, ev_y_pred)
#     scores.append(score)

# print(np.mean(scores))

In [42]:
from sklearn.model_selection import KFold 

time_sta = time.perf_counter()

kf = KFold(n_splits = 4, shuffle=True, random_state=0)

dcv = lgb.Dataset(X_train, label=y_train)  # クロスバリデーション用
verbose_eval = 0  # この数字を1にすると学習時のスコア推移がコマンドライン表示される

# early_stoppingを指定してLightGBMをクロスバリデーション
print(datetime.datetime.now())
verbose_eval = 0
cv_result = lgb.cv(params, dcv,
                num_boost_round=10000,  # 最大学習サイクル数。early_stopping使用時は大きな値を入力
                folds=kf,
                callbacks=[lgb.early_stopping(stopping_rounds=10, 
                                verbose=True), # early_stopping用コールバック関数
                           lgb.log_evaluation(verbose_eval)] # コマンドライン出力用コールバック関数
                )
print(datetime.datetime.now())

time_end = time.perf_counter()
print(time_end- time_sta)

2023-10-13 08:54:29.110370


C:\Users\s-heya\Anaconda3\lib\site-packages\lightgbm\engine.py:674: UserWarning: Found 'num_iterations' in params. Will use it instead of 'num_boost_round' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'num_boost_round' argument")


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[5956]	cv_agg's valid l1: 0.0860081 + 0.000415986
2023-10-13 09:07:17.883558
768.7732886


In [43]:
print('MSE mean={}'.format(cv_result['l1-mean'][-1]))
print(cv_result)

KeyError: 'l1-mean'

In [ ]:
import shap

exp = shap.TreeExplainer(model=model, feature_perturbation='tree_path_dependent')

In [ ]:

print(datetime.datetime.now())
time_sta = time.perf_counter()

shap_values = exp(X_valid.head(5))

time_end = time.perf_counter()
print(datetime.datetime.now())

print(time_end- time_sta)

In [ ]:

shap.plots.bar(shap_values=shap_values, max_display=16)

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
bins = np.linspace(-2, 2, 50)

df = X_valid_base.copy()
df['差'] = df['取引価格（総額）_log']-df['pred_取引価格（総額）_log']


plt.hist(df['差'], bins, alpha = 0.5, label='1')

plt.legend(loc='upper left')

plt.show()

# --
scatter_plt = plt.figure()

ax = scatter_plt.add_subplot(1,1,1)
ax.scatter(df['pred_取引価格（総額）_log'],df['取引価格（総額）_log'])

ax.set_xlabel('予測値')
ax.set_ylabel('実測値')

ax.set_xlim(4, 10)
ax.set_ylim(4, 10)

plt.rcParams["figure.figsize"] = [10,5]  
ax.grid(which = "major", axis = "x", color = "blue", alpha = 0.8,
        linestyle = "--", linewidth = 1)
ax.grid(which = "major", axis = "y", color = "green", alpha = 0.8,
        linestyle = "--", linewidth = 1)

scatter_plt.show()


print(df['差'].describe())

In [ ]:
scatter_plt = plt.figure()
plt.rcParams["figure.figsize"] = [10,5]  
ax = scatter_plt.add_subplot(1,1,1)
ax.scatter(y_valid, y_valid-X_valid_base['pred_取引価格（総額）_log'])

ax.set_xlabel('実績値')
ax.set_ylabel('残差')

# ax.set_xlim(0, 10)
# ax.set_ylim(0, 10)

ax.grid(which = "major", axis = "x", color = "blue", alpha = 0.8,
        linestyle = "--", linewidth = 1)
ax.grid(which = "major", axis = "y", color = "green", alpha = 0.8,
        linestyle = "--", linewidth = 1)



In [ ]:
scatter_plt = plt.figure()
plt.rcParams["figure.figsize"] = [10,5]  
ax = scatter_plt.add_subplot(1,1,1)
ax.scatter(X_valid_base['pred_取引価格（総額）_log'], y_valid-X_valid_base['pred_取引価格（総額）_log'])

ax.set_xlabel('予測値')
ax.set_ylabel('残差')

ax.set_xlim(5, 9)
# ax.set_ylim(0, 10)

ax.grid(which = "major", axis = "x", color = "blue", alpha = 0.8,
        linestyle = "--", linewidth = 1)
ax.grid(which = "major", axis = "y", color = "green", alpha = 0.8,
        linestyle = "--", linewidth = 1)



In [ ]:
X_valid_base.columns

# パラメーターチューニング

In [ ]:
model.params

In [ ]:
import optuna.integration.lightgbm as lgb

params = {
    'objective': 'regression_l1',  # 最小化させるべき損失関数
    'metric': 'mae',  # 学習時に使用する評価指標(early_stoppingの評価指標にも同じ値が使用される)
    'random_state': 0,  # 乱数シード
    'boosting_type': 'gbdt',  # boosting_type
    'n_estimators': 10000,  # 最大学習サイクル数。early_stopping使用時は大きな値を入力
    'verbose': -1,  # これを指定しないと`No further splits with positive gain, best gain: -inf`というWarningが表示される
    'n_jobs':16,
    'early_stopping_round': 10,  # ここでearly_stoppingを指定
    'force_col_wise':'true',
    'max_depth':7
}

lgb_train = lgb.Dataset(X_train,y_train,categorical_feature=categorical_features)
lgb_eval  = lgb.Dataset(X_valid,y_valid,reference=lgb_train,categorical_feature=categorical_features)

# モデルの学習
best_params, tuning_history = dict(), list()
model = lgb.train(params, lgb_train, valid_sets=[lgb_train,lgb_eval],verbose_eval=0)

In [ ]:
# import optuna
# import time
# start = time.time()

# fit_params = {
#     'eval_set': [(X_valid, y_valid)]  # early_stopping用のデータeval_setを渡す
#     }

# # ベイズ最適化時の評価指標算出メソッド
# def bayes_objective(trial):
#     params = {
#         'objective': 'regression_l1',  # 最小化させるべき損失関数
#         'metric': 'mae',  # 学習時に使用する評価指標(early_stoppingの評価指標にも同じ値が使用される)
#         'random_state': 0,  # 乱数シード
#         'boosting_type': 'gbdt',  # boosting_type
#         'n_estimators': 10000,  # 最大学習サイクル数。early_stopping使用時は大きな値を入力
#         'verbose': -1,  # これを指定しないと`No further splits with positive gain, best gain: -inf`というWarningが表示される
#         'early_stopping_round': 10,  # ここでearly_stoppingを指定
#         'force_col_wise':'true',
#         'n_jobs':16,
#         'max_depth':trial.suggest_int('max_depth', 2, 30)
#     }
    
#     lgb_train = lgb.Dataset(X_train,y_train,categorical_feature=categorical_features)
#     lgb_eval  = lgb.Dataset(X_valid,y_valid,reference=lgb_train,categorical_feature=categorical_features)
    
#     model = lgb.train(params, lgb_train, valid_sets=[lgb_train,lgb_eval],verbose_eval=0)
    
#     va_y_pred = model.predict(X_valid)
    
#     score = mean_absolute_error(y_valid, va_y_pred)
    
#     print(f'経過時間{time.time() - start}秒')

#     return score

# # ベイズ最適化を実行
# study = optuna.create_study(direction='maximize',
#                             sampler=optuna.samplers.TPESampler(seed=0))
# study.optimize(bayes_objective, n_trials=40)

# # 最適パラメータの表示と保持
# best_params = study.best_trial.params
# best_score = study.best_trial.value
# print(f'最適パラメータ {best_params}\nスコア {best_score}')
# print(f'所要時間{time.time() - start}秒')

# 以降は作業スペース

In [ ]:
sys.exit()

In [ ]:
df = X_valid_base.copy()
df['差'] = y_valid-df['pred_取引価格（総額）_log']


In [ ]:
len(df[(df['差'] <-0.17)])

In [ ]:
len(df[(df['差'] >0.17)])